In [1]:

import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import subprocess
import pickle
from scipy.stats import describe
from tqdm import tqdm

from astropy.io import fits

from lvmdrp.core import image
from lvmdrp.functions import imageMethod, rssMethod

import seaborn as sns

sns.set(context="talk", style="ticks", palette="husl", color_codes=True)

import warnings
warnings.filterwarnings("ignore")


def read_nist_reference(filename, wl_window):
    wli, wlf = wl_window
    ref_nist_or = pd.read_csv(
        filename, sep="\s+", comment="#").set_index("Wavelength")
    ref_nist_or = ref_nist_or.loc[(
        wli <= ref_nist_or.index) & (ref_nist_or.index <= wlf)]

    del_wl = np.diff(ref_nist_or.index).min()
    new_wl = np.unique(
        np.arange(wli, wlf+del_wl, del_wl).tolist()+ref_nist_or.index.to_list())
    ref_nist_rs = ref_nist_or.reindex(new_wl)

    return ref_nist_or, ref_nist_rs


In [2]:
# data root path
LVM_DEV = os.path.expandvars("$LVMHUB")
LVM_ROOT = os.path.join(LVM_DEV, "data.sdss5.org/sas/sdsswork/data/lvm")
LVM_DATA_LAB = os.path.join(LVM_ROOT, "lab", "59898")

# Spec 3 6.2: arcs 6413 - 6772
# Spec 3 6.2: flats 6775 - 6845
cont_idxs = np.arange(6775, 6845+1, 1, dtype=int)

frame_paths = sorted([os.path.join(root,file) for root, _, files in os.walk(LVM_DATA_LAB) for file in files if file.endswith(".fits.gz") and int(file.replace(".fits.gz", "").split("-")[-1]) in cont_idxs])
print(f"frames matched in lab data path: {len(frame_paths)}")


frames matched in lab data path: 213


In [3]:
fiberflat_path = "_data-lvm/fiberflats-spec3"
if not os.path.isdir(fiberflat_path):
    os.makedirs(fiberflat_path, exist_ok=True)

ccds = ["b3", "r3", "z3"]

cont_paths = list(reversed(frame_paths))

for cont_path in tqdm(cont_paths, total=len(cont_paths), desc="processing continuum frames", ascii=True, unit="frame"):

    hdr = fits.getheader(cont_path)
    if hdr["IMAGETYP"] != "object": continue

    cont_name = os.path.basename(cont_path).replace(".fits.gz", "")

    if os.path.isfile(f"{fiberflat_path}/{cont_name}.ms.fits"): continue

    try:    
        imageMethod.preprocRawFrame_drp(
            in_image=cont_path,
            out_image=f"{fiberflat_path}/{cont_name}.pre.fits",
            boundary_x="1,2040",
            boundary_y="1,2040",
            positions="00,10,01,11",
            orientation="S,S,S,S"
        )

        # imageMethod.LACosmic_drp(
        #     image=f"{fiberflat_path}/{cont_name}.pre.fits",
        #     out_image=f"{fiberflat_path}/{cont_name}.cosmic.fits",
        #     increase_radius=1, flim="1.3", parallel='5'
        # )

        imageMethod.findPeaksAuto_drp(
            image=f"{fiberflat_path}/{cont_name}.pre.fits",
            out_peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks",
            disp_axis="X", threshold="5000", slice="3696", nfibers="41", median_box="1", median_cross="1", method="hyperbolic", init_sigma="0.5", verbose=0
        )

        imageMethod.tracePeaks_drp(
            image=f"{fiberflat_path}/{cont_name}.pre.fits",
            peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks", trace_out=f"{fiberflat_path}/{cont_name}.trc.fits",
            steps=30, method="hyperbolic", threshold_peak=50, poly_disp=5, coadd=30, verbose=0
        )

        imageMethod.subtractStraylight_drp(
            image=f"{fiberflat_path}/{cont_name}.pre.fits",
            trace=f"{fiberflat_path}/{cont_name}.trc.fits",
            stray_image=f"{fiberflat_path}/{cont_name}.back.fits",
            clean_image=f"{fiberflat_path}/{cont_name}.stray.fits",
            aperture=40, poly_cross=2, smooth_gauss=30
        )

        imageMethod.traceFWHM_drp(
            image=f"{fiberflat_path}/{cont_name}.stray.fits",
            trace=f"{fiberflat_path}/{cont_name}.trc.fits",
            fwhm_out=f"{fiberflat_path}/{cont_name}.fwhm.fits",
            blocks=32, steps=30, coadd=20, threshold_flux=50.0, poly_disp=5, clip="1.5,4.0"
        )

        imageMethod.extractSpec_drp(
            image=f"{fiberflat_path}/{cont_name}.stray.fits",
            trace=f"{fiberflat_path}/{cont_name}.trc.fits",
            out_rss=f"{fiberflat_path}/{cont_name}.ms.fits",
            fwhm=f"{fiberflat_path}/{cont_name}.fwhm.fits",
            parallel=5, method="optimal"
        )
    except:
        continue

processing continuum frames: 100%|##########| 213/213 [3:24:54<00:00, 57.72s/frame]    
